In [1]:
%load_ext autoreload
%autoreload 2

import os
from os import path
import sys
cwd = os.getcwd().split(os.path.sep)

# point to the git repository
while cwd[-1] != "ExplanationPairSentencesTasks":
    os.chdir("..")
    cwd = os.getcwd().split(os.path.sep)
print(f">> current directory : {os.getcwd()}")

# add the root directory
sys.path.append(os.path.join(os.getcwd(), "src"))

# cache and data cache
cache_path = path.join(os.getcwd() ,'.cache')
dataset_path = path.join(cache_path, 'dataset')
log_path = path.join(cache_path, 'logs')
model_path = path.join(cache_path, 'models')
emb_path = path.join(cache_path, 'embeddings')
print(f">> cache path : {cache_path}")
print(f">> model path : {model_path}")
print(f">> dataset path : {dataset_path}")
print(f">> logs path : {log_path}")
print(f">> emb path : {emb_path}")

# import the different modules
from src.data_module.hatexplain import CLSTokenHateXPlainDM
from src.data_module.esnli import CLSTokenESNLIDM
from src.data_module.yelp_hat import CLSTokenYelpHat50DM, CLSTokenYelpHatDM
from pur_attention_key_reg import AttitModel

from notebooks.attention_based.utils.attention_embeddings import *
from notebooks.attention_based.utils.one_step_window import attention_metrics_res as one_step_window

# external librairies
import torch
import seaborn as sns
sns.set_theme()

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f">> device : {DEVICE}")

>> current directory : C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks
>> cache path : C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache
>> model path : C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\models
>> dataset path : C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\dataset
>> logs path : C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\logs
>> emb path : C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings
>> device : cuda


# Hatexplain

In [3]:
res_array = []
res_rolling_array = []

dm_kwargs = dict(cache_path=dataset_path,
                 batch_size=32,
                 num_workers=0,
                 n_data=-1
                 )

dm = CLSTokenHateXPlainDM(**dm_kwargs)
dm.prepare_data()
dm.setup(stage="test")

model_args = dict(
        cache_path=model_path,
        mode="exp",
        vocab=dm.vocab,
        lambda_entropy=0,
        lambda_supervise=0,
        lambda_lagrange=0,
        pretrained_vectors="glove.840B.300d",
        num_layers=1,
        num_heads=1,
        d_embedding=300,
        data="hatexplain",
        num_class=dm.num_class,
        opt="adam"
)

for l in range(5):

    model_args["num_layers"] = l+1
    ckp = os.path.join(log_path, "PurAttention", f"run=0_hatexplain_l=0{l+1}_h=1_adam", "checkpoints", "best.ckpt")
    hparams = os.path.join(log_path, "PurAttention", f"run=0_hatexplain_l=0{l+1}_h=1_adam", "hparams.yaml")

    # the model
    model = AttitModel.load_from_checkpoint(ckp, hparams_file=hparams, **model_args)
    model = model.eval()

    res = attention_metrics_res(model.to(DEVICE), dm, cache = os.path.join(emb_path, f"matrix_e_hatexplain_{l+1}.pt"), nb_data = 2000)
    res_array.append(res)

    """res_windows = one_step_window(model.to(DEVICE), dm, nb_data = 900)
    res_rolling_array.append(res_windows)"""

C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


load the matrix at the location C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings\matrix_e_hatexplain_1.pt ... loading done !


proceed the cosine map: : 61it [00:05, 12.09it/s]


test passed : torch.Size([26785])



C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


...

test passed : torch.Size([26785])



C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


load the matrix at the location C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings\matrix_e_hatexplain_5.pt ... loading done !


proceed the cosine map: : 61it [00:01, 37.70it/s]

test passed : torch.Size([26785])



In [4]:
for i,res in enumerate(res_array):
    print()
    print(f"--- {i+1} layer ---")
    print()
    dict_print(res)


--- 1 layer ---

AUC - cos  :  0.6453655695112541
AUC - attention  :  0.7085374490171078

--- 2 layer ---

AUC - cos  :  0.642540197764027
AUC - attention  :  0.5190543639619913

--- 3 layer ---

AUC - cos  :  0.6440181538479587
AUC - attention  :  0.5199078098001578

--- 4 layer ---

AUC - cos  :  0.6561134749328668
AUC - attention  :  0.5146393865783421

--- 5 layer ---

AUC - cos  :  0.6570189931225057
AUC - attention  :  0.5457106851987926


# Yelphat

In [4]:
res_array = []
res_rolling_array = []

dm_kwargs = dict(cache_path=dataset_path,
                 batch_size=32,
                 num_workers=0,
                 n_data=-1
                 )

dm = CLSTokenYelpHat50DM(**dm_kwargs)
dm.prepare_data()
dm.setup(stage="test")

model_args = dict(
        cache_path=model_path,
        mode="exp",
        vocab=dm.vocab,
        lambda_entropy=0,
        lambda_supervise=0,
        lambda_lagrange=0,
        pretrained_vectors="glove.840B.300d",
        num_layers=1,
        num_heads=1,
        d_embedding=300,
        data="yelphat",
        num_class=dm.num_class,
        opt="adam"
)

for l in range(5):
    model_args["num_layers"] = l+1
    ckp = os.path.join(log_path, "PurAttention", f"run=0_yelphat50_l=0{l+1}_h=1_adam", "checkpoints", "best.ckpt")
    hparams = os.path.join(log_path, "PurAttention", f"run=0_yelphat50_l=0{l+1}_h=1_adam", "hparams.yaml")

    # the model
    model = AttitModel.load_from_checkpoint(ckp, hparams_file=hparams, **model_args)
    model = model.eval()

    res = attention_metrics_res(model.to(DEVICE), dm, cache = os.path.join(emb_path, f"matrix_e_yelphat50_{l+1}.pt"), nb_data = 2000)
    res_array.append(res)

    """res_windows = one_step_window(model.to(DEVICE), dm, nb_data = 900)
    res_rolling_array.append(res_windows)"""

C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


10
load the matrix at the location C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings\matrix_e_yelphat50_1.pt ... loading done !


proceed the cosine map: : 10it [00:00, 20.88it/s]


test passed : torch.Size([17402])



C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


...

test passed : torch.Size([17402])



C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


10
load the matrix at the location C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings\matrix_e_yelphat50_5.pt ... loading done !


proceed the cosine map: : 10it [00:00, 20.39it/s]

test passed : torch.Size([17402])



In [5]:
for i,res in enumerate(res_array):
    print()
    print(f"--- {i+1} layer ---")
    print()
    dict_print(res)


--- 1 layer ---

AUC - cos  :  0.6099390794017335
AUC - attention  :  0.7116495788771671

--- 2 layer ---

AUC - cos  :  0.593299380195236
AUC - attention  :  0.600224247561349

--- 3 layer ---

AUC - cos  :  0.6157801018581223
AUC - attention  :  0.6256304762209922

--- 4 layer ---

AUC - cos  :  0.6065181299778823
AUC - attention  :  0.5497422215721981

--- 5 layer ---

AUC - cos  :  0.6049565216323993
AUC - attention  :  0.5388016176212471


# E-SNLI

In [24]:
res_array = []
res_rolling_array = []

dm_kwargs = dict(cache_path=dataset_path,
                 batch_size=32,
                 num_workers=0,
                 n_data=-1
                 )

dm = CLSTokenESNLIDM(**dm_kwargs)
dm.prepare_data()
dm.setup(stage="test")

model_args = dict(
        cache_path=model_path,
        mode="exp",
        vocab=dm.vocab,
        lambda_entropy=0,
        lambda_supervise=0,
        lambda_lagrange=0,
        pretrained_vectors="glove.840B.300d",
        num_layers=1,
        num_heads=1,
        d_embedding=300,
        data="esnli",
        num_class=dm.num_class,
        opt="adam"
)

for l in range(2):
    model_args["num_layers"] = l+1
    ckp = os.path.join(log_path, "PurAttention", f"run=0_esnli_l=0{l+1}_h=1_adam", "checkpoints", "best.ckpt")
    hparams = os.path.join(log_path, "PurAttention", f"run=0_esnli_l=0{l+1}_h=1_adam", "hparams.yaml")

    # the model
    model = AttitModel.load_from_checkpoint(ckp, hparams_file=hparams, **model_args)
    model = model.eval()

    res = attention_metrics_res(model.to(DEVICE), dm, cache = os.path.join(emb_path, f"matrix_e_esnli_{l+1}.pt"), nb_data = 2000)
    res_array.append(res)

    """res_windows = one_step_window(model.to(DEVICE), dm, nb_data = 900)
    res_rolling_array.append(res_windows)"""

C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


0
load the matrix at the location C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings\matrix_e_esnli_1.pt ... loading done !


proceed the cosine map: : 93it [00:02, 41.71it/s]


done !
test passed : torch.Size([47087])



C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\loicf\miniconda3\envs\nlp\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


0
load the matrix at the location C:\Users\loicf\Documents\IRISA\ExplanationPairSentencesTasks\.cache\embeddings\matrix_e_esnli_2.pt ... loading done !


proceed the cosine map: : 93it [00:01, 48.00it/s]

done !
test passed : torch.Size([47087])



In [25]:
for i,res in enumerate(res_array):
    print()
    print(f"--- {i+1} layer ---")
    print()
    dict_print(res)


--- 1 layer ---

AUC - cos  :  0.5674373093624985
AUC - attention  :  0.6013851855942991

--- 2 layer ---

AUC - cos  :  0.6887597757945834
AUC - attention  :  0.5788340498030906
